# Import libs

In [1]:
from fastkml import KML
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import pandas as pd
import numpy as np

from pykrige.ok import OrdinaryKriging
from pyproj import Transformer

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.colors import Normalize

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import networkx as nx

/home/ric/Documents/RandomCode/Rizzo/venv/lib/python3.10/site-packages/fastkml/config.py:28: UserWarning: Package `lxml` missing. Pretty print will be disabled
  warnings.warn("Package `lxml` missing. Pretty print will be disabled")


# Define helper functions

In [2]:
def kml_to_geodataframe(kml_file):
    with open(kml_file, 'rb') as f:
        kml_data = f.read()

    k = KML()
    k.from_string(kml_data)

    features = list(k.features())[0].features()

    # Extract the coordinates and properties from the KML features
    coordinates = []
    properties = []

    for feature in features:
        coords = (feature.geometry.x, feature.geometry.y)
        props = {'name': feature.name}
        coordinates.append(coords)
        properties.append(props)

    # Create a GeoDataFrame from the extracted data
    geometry = [Point(coord) for coord in coordinates]
    gdf = gpd.GeoDataFrame(properties, geometry=geometry)

    return gdf

# Function to convert lat, lon to UTM
def latlon_to_utm(lats, lons):
    transformer = Transformer.from_crs("epsg:4326", "epsg:32613")  # Updated to EPSG:32613
    return transformer.transform(lats, lons)

# Function to convert UTM to lat, lon
def utm_to_latlon(xs, ys):
    transformer = Transformer.from_crs("epsg:32613", "epsg:4326")  # Updated to EPSG:32613 EPSG:26913
    return transformer.transform(xs, ys)

def dfs(node, visited, G):
    i, j = node
    visited.add(node)
    
    for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        neighbor_node = (i + di, j + dj)
        if neighbor_node in G and neighbor_node not in visited:
            G.add_edge(node, neighbor_node)
            dfs(neighbor_node, visited, G)

def next_state(G, val):
    new_val = {node: 0 for node in G.nodes}
    
    for node in G.nodes:
        out_edges = list(G.out_edges(node))
        if not out_edges:
            new_val[node] += val[node]  # Keep the value if there are no outgoing edges
        else:
            # Distribute the value equally among the adjacent nodes
            distributed_value = val[node] / len(out_edges)
            for _, neighbor_node in out_edges:
                new_val[neighbor_node] += distributed_value
                
    return new_val


# Load data

In [3]:
# Read the shapefile
river_data = gpd.read_file('../src/data/santiago/rio_santiago_prueba.shp')
river_data = river_data.to_crs("epsg:4326")

# Read the CSV file
measured_variables = pd.read_csv('../src/data/rizo20092013update.csv')
measured_variables = measured_variables.drop('Unnamed: 0', axis=1)

# Read the KML file
sampling_stations = kml_to_geodataframe('../src/data/muestreo.csv.kml')
sampling_stations = sampling_stations[:10].copy()
sampling_stations['name'] = [f'RS-{i:02d}' for i in range(1,11)]

In [4]:
# Extract the coordinates of the sampling stations
coordinates = sampling_stations.geometry.apply(lambda point: [point.y, point.x]).tolist()

# Get chosen variable df
variable_name = 'Coliformes totales'
var = measured_variables[['fecha', 'idPuntoMuestreo', variable_name]]

# Apply to sampling stations
values = []
for i in range(1,11):
    val = (var[var['idPuntoMuestreo'] == np.float64(i)].dropna())['Coliformes totales'].astype(np.float64).values.mean()
    values.append(val)

In [6]:
# Prepare the data for Kriging
lats, lons = zip(*coordinates)

# Convert lat, lon to UTM
lats_utm, lons_utm = latlon_to_utm(lats, lons)

# Calculate the number of grid cells in UTM coordinates
cell_size = 500  # 5000 meters
lon_range_utm = max(lons_utm) - min(lons_utm)
lat_range_utm = max(lats_utm) - min(lats_utm)

lon_cells = int(np.ceil(lon_range_utm / cell_size))
lat_cells = int(np.ceil(lat_range_utm / cell_size))

grid_lon_utm_ls = np.linspace(min(lons_utm), min(lons_utm) + lon_cells * cell_size, num=lon_cells)
grid_lat_utm_ls = np.linspace(min(lats_utm), min(lats_utm) + lat_cells * cell_size, num=lat_cells)

grid_lon_utm, grid_lat_utm = np.meshgrid(grid_lon_utm_ls, grid_lat_utm_ls)
# Convert the UTM grid back to lat, lon
grid_lat, grid_lon = utm_to_latlon(grid_lat_utm, grid_lon_utm)

# Grid dimension
geo_grid_shape = grid_lat.T.shape

In [7]:
# Obtain river geodata (lat,lon)
river_lats = []
river_lons = []

for line in list(river_data.geometry[0].geoms)[::-1]:
    for point in line.coords:
        river_lats.append(point[1])
        river_lons.append(point[0])

start = 14000
stop = -150

river_lats = river_lats[start:stop]
river_lons = river_lons[start:stop]

# Generate river mask
# Create a LineString from river_lons and river_lats
river_line = LineString(list(zip(river_lons, river_lats)))

# Create a mask for the meshgrid based on river_line
mask = np.zeros(geo_grid_shape, dtype=bool)

for i in range(geo_grid_shape[0]-1):
    for j in range(geo_grid_shape[1]-1):
        cell_polygon = Polygon([
            (grid_lon.T[i, j], grid_lat.T[i, j]),
            (grid_lon.T[i + 1, j], grid_lat.T[i + 1, j]),
            (grid_lon.T[i + 1, j + 1], grid_lat.T[i + 1, j + 1]),
            (grid_lon.T[i, j + 1], grid_lat.T[i, j + 1])
        ])

        mask[i, j] = not river_line.intersects(cell_polygon)

# Handle last row
for j in range(geo_grid_shape[1] - 1):
    cell_polygon = Polygon([
        (grid_lon.T[-2, j], grid_lat.T[-2, j]),
        (grid_lon.T[-1, j], grid_lat.T[-1, j]),
        (grid_lon.T[-1, j + 1], grid_lat.T[-1, j + 1]),
        (grid_lon.T[-2, j + 1], grid_lat.T[-2, j + 1])
    ])
    mask[-1, j] = not river_line.intersects(cell_polygon)

# Handle last column
for i in range(geo_grid_shape[0] - 1):
    cell_polygon = Polygon([
        (grid_lon.T[i, -2], grid_lat.T[i, -2]),
        (grid_lon.T[i + 1, -2], grid_lat.T[i + 1, -2]),
        (grid_lon.T[i + 1, -1], grid_lat.T[i + 1, -1]),
        (grid_lon.T[i, -1], grid_lat.T[i, -1])
    ])
    mask[i, -1] = not river_line.intersects(cell_polygon)

# Handle bottom right corner cell
cell_polygon = Polygon([
    (grid_lon.T[-2, -2], grid_lat.T[-2, -2]),
    (grid_lon.T[-1, -2], grid_lat.T[-1, -2]),
    (grid_lon.T[-1, -1], grid_lat.T[-1, -1]),
    (grid_lon.T[-2, -1], grid_lat.T[-2, -1])
])
mask[-1, -1] = not river_line.intersects(cell_polygon)

In [11]:
# Load high-resolution features for cartopy
states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='10m',
    facecolor='none')

roads = cfeature.NaturalEarthFeature(
    category='cultural',
    name='roads',
    scale='10m',
    facecolor='none')

# Create flow graph for given resolution

In [12]:
# Create an empty graph
G = nx.DiGraph()

# Add nodes to the graph based on the mask
for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
        if not mask[i, j]:
            node = (i, j)
            G.add_node(node)

# Find the southernmost grid cell
south_node = None
min_lat = np.inf

for node in G.nodes:
    i, j = node
    lat = grid_lat.T[i, j]
    if lat < min_lat:
        min_lat = lat
        south_node = node

# Perform DFS traversal from the southernmost grid cell
visited = set()
dfs(south_node, visited, G)

monitoring_station = []
for lat, lon in zip(lats, lons):
    closest_node = None
    min_distance = np.inf
    
    for node in G.nodes:
        i, j = node
        node_lat, node_lon = grid_lat.T[i, j], grid_lon.T[i, j]
        distance = np.sqrt((node_lat - lat)**2 + (node_lon - lon)**2)
        
        if distance < min_distance:
            min_distance = distance
            closest_node = node
    
    monitoring_station.append(closest_node)

pos = {node: (grid_lon.T[node], grid_lat.T[node]) for node in G.nodes}  # Define positions based on lon and lat

# LACY-eng test 

In [ ]:
n_steps = 50
now_values = values

for _ in range(n_steps):
    # -------- Generate OK model for initial values --------

    print('Calculating OK')
    ok = OrdinaryKriging(lons_utm, lats_utm, now_values, variogram_model='linear')
    # Estimate values and variance on the grid in UTM coordinates
    z_estimated, _ = ok.execute('grid', grid_lon_utm_ls, grid_lat_utm_ls)
    z_masked = np.ma.array(z_estimated.T, mask=mask)
    val = {node: z_masked[node] for node in G.nodes}

    # -------- Plot values for current state --------

    print('Generating geoplot')
    # Define the coordinate system
    projection = ccrs.PlateCarree()

    # Create the map figure
    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': projection})

    # Set the map extent based on lat-lon range
    lon_min, lon_max = np.min(grid_lon), np.max(grid_lon)
    lat_min, lat_max = np.min(grid_lat), np.max(grid_lat)
    ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=projection)

    # Add map features
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')

    # Add high-resolution features
    ax.add_feature(states_provinces, edgecolor='gray')
    ax.add_feature(roads, edgecolor='black', linestyle='--')

    # Plot the masked data on the map
    mesh = ax.pcolormesh(grid_lon.T, grid_lat.T, z_masked, cmap='jet', alpha=0.95, transform=projection)
    plt.colorbar(mesh, ax=ax, label='Estimated values (NPM/100ml)')

    ax.plot(river_lons, river_lats, c='blue', linewidth=4, transform=projection, alpha=0.10)
    #scatter = ax.scatter(lons, lats, c=values, edgecolors='k', cmap='jet', s=50, zorder=2, transform=projection)
    scatter = ax.scatter(lons, lats, c='violet', edgecolors='k', marker='v', s=150, zorder=2, transform=projection)

    plt.xlabel('Long °')
    plt.ylabel('Lat °')
    plt.title('OK L2 - Linear')
    plt.show()

    #-------- Generate new state --------
    
    print('generating timestep')
    for _ in range(50):
        val = next_state(G,val)

    now_values = [val[node]*0.75 for node in monitoring_station] 
    #now_values = [val[node] for node in G.nodes if node in monitoring_station] 

In [ ]:
n_steps = 10
now_values = values
linear = True

for _ in range(n_steps):
    # -------- Generate OK model for initial values --------

    print('Calculating OK')
    if linear:
        ok = OrdinaryKriging(lons_utm, lats_utm, now_values, variogram_model='linear',enable_plotting=True)
        linear = False
    else:
        ok = OrdinaryKriging(lons_utm, lats_utm, now_values, variogram_model='gaussian',enable_plotting=True)
    # Estimate values and variance on the grid in UTM coordinates
    z_estimated, _ = ok.execute('grid', grid_lon_utm_ls, grid_lat_utm_ls)
    z_masked = np.ma.array(z_estimated.T, mask=mask)
    val = {node: z_estimated.T[node] for node in G.nodes}

    # -------- Plot values for current state --------

    print('Generating graph plot')
    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(10, 8))

    # Draw the directed graph
    cmap = get_cmap('jet')
    norm = Normalize(vmin=np.nanmin(z_masked), vmax=np.nanmax(z_masked))

    #mon_s_colors = ['violet' if node in monitoring_station else cmap(norm(val[node])) for node in G.nodes]
    #mon_s_colors = [cmap(norm(val[node])) if node in monitoring_station else 'gray' for node in G.nodes]
    mon_s_colors = [cmap(norm(val[node])) for node in G.nodes]
    mon_s_sizes = [100 if node in monitoring_station else 45 for node in G.nodes]
    nx.draw_networkx_nodes(G, pos, ax=ax, node_color=mon_s_colors, node_size=mon_s_sizes)
    nx.draw_networkx_edges(G, pos, ax=ax)

    ax.pcolormesh(grid_lon.T, grid_lat.T, z_estimated.T, cmap='jet', alpha=0.45)
    #ax.colorbar(label='Estimated values (NPM/100ml)')
    #ax.plot(river_lons,river_lats, c='red', linewidth=4)
    ax.scatter(lons, lats, c=now_values, edgecolors='k', cmap='jet', s=50, zorder=2)

    # Set axis labels and title
    plt.xlabel('Long °')
    plt.ylabel('Lat °')
    plt.title('Directed Graph of the River')

    # Show the plot
    plt.show()

    #-------- Generate new state --------
    
    print('generating timestep')
    val = next_state(G,val)
    now_values = [val[node] for node in monitoring_station] 

    #-------- Sanity check --------

    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(10, 8))

    # Draw the directed graph
    cmap = get_cmap('jet')
    norm = Normalize(vmin=np.nanmin(z_masked), vmax=np.nanmax(z_masked))

    #mon_s_colors = ['violet' if node in monitoring_station else cmap(norm(val[node])) for node in G.nodes]
    #mon_s_colors = [cmap(norm(val[node])) if node in monitoring_station else 'gray' for node in G.nodes]
    mon_s_colors = [cmap(norm(val[node])) for node in G.nodes]
    mon_s_sizes = [100 if node in monitoring_station else 45 for node in G.nodes]
    nx.draw_networkx_nodes(G, pos, ax=ax, node_color=mon_s_colors, node_size=mon_s_sizes)
    nx.draw_networkx_edges(G, pos, ax=ax)

    ax.pcolormesh(grid_lon.T, grid_lat.T, z_estimated.T, cmap='jet', alpha=0.45)
    #ax.colorbar(label='Estimated values (NPM/100ml)')
    #ax.plot(river_lons,river_lats, c='red', linewidth=4)
    ax.scatter(lons, lats, c=now_values, edgecolors='k', cmap='jet', s=50, zorder=2)

    # Set axis labels and title
    plt.xlabel('Long °')
    plt.ylabel('Lat °')
    plt.title('Directed Graph of the River')

    # Show the plot
    plt.show()

In [ ]:
n_steps = 10
now_values = values
#linear = True

for _ in range(n_steps):
    # -------- Generate OK model for initial values --------

    print('Calculating OK')
    for _ in range(10):
        #print('Calculating OK')
        ok = OrdinaryKriging(lons_utm, lats_utm, now_values, variogram_model='gaussian')

        # Estimate values and variance on the grid in UTM coordinates
        z_estimated, _ = ok.execute('grid', grid_lon_utm_ls, grid_lat_utm_ls)
        z_masked = np.ma.array(z_estimated.T, mask=mask)
        val = {node: z_estimated.T[node] for node in G.nodes}

        #print('generating timestep')
        val = next_state(G,val)
        now_values = [val[node] for node in monitoring_station]

    # -------- Plot values for current state --------

    print('Generating graph plot')
    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(10, 8))

    # Draw the directed graph
    cmap = get_cmap('jet')
    norm = Normalize(vmin=np.nanmin(z_masked), vmax=np.nanmax(z_masked))

    #mon_s_colors = ['violet' if node in monitoring_station else cmap(norm(val[node])) for node in G.nodes]
    #mon_s_colors = [cmap(norm(val[node])) if node in monitoring_station else 'gray' for node in G.nodes]
    mon_s_colors = [cmap(norm(val[node])) for node in G.nodes]
    mon_s_sizes = [100 if node in monitoring_station else 45 for node in G.nodes]
    nx.draw_networkx_nodes(G, pos, ax=ax, node_color=mon_s_colors, node_size=mon_s_sizes)
    nx.draw_networkx_edges(G, pos, ax=ax)

    ax.pcolormesh(grid_lon.T, grid_lat.T, z_estimated.T, cmap='jet', alpha=0.45)
    #ax.colorbar(label='Estimated values (NPM/100ml)')
    #ax.plot(river_lons,river_lats, c='red', linewidth=4)
    ax.scatter(lons, lats, c=now_values, edgecolors='k', cmap='jet', s=50, zorder=2)

    # Set axis labels and title
    plt.xlabel('Long °')
    plt.ylabel('Lat °')
    plt.title('Directed Graph of the River')

    # Show the plot
    plt.show()

    #-------- Generate new state --------
    
    print('generating timestep')
    val = next_state(G,val)
    now_values = [val[node] for node in monitoring_station] 

In [13]:
n_steps = 10
now_values = values

time_k = []

for _ in range(n_steps):
    # -------- Generate OK model for initial values --------

    print('Calculating OK')
    ok = OrdinaryKriging(lons_utm, lats_utm, now_values, variogram_model='linear')
    # Estimate values and variance on the grid in UTM coordinates
    z_estimated, _ = ok.execute('grid', grid_lon_utm_ls, grid_lat_utm_ls)
    z_masked = np.ma.array(z_estimated.T, mask=mask)
    val = {node: z_masked[node] for node in G.nodes}

    # -------- Plot values for current state --------

    density_lat = []
    density_lon = []
    density_z = []

    for i in range(z_masked.shape[0]):
        for j in range(z_masked.shape[1]):
            if not mask[i, j]:
                density_lat.append(grid_lat.T[i,j])
                density_lon.append(grid_lon.T[i,j])
                density_z.append(z_masked.data[i,j])

    time_k.append((density_lat,density_lon,density_z))

    #-------- Generate new state --------
    
    print('generating timestep')
    for _ in range(50):
        val = next_state(G,val)

    now_values = [val[node]*0.75 for node in monitoring_station] 
    #now_values = [val[node] for node in G.nodes if node in monitoring_station] 

Calculating OK
generating timestep
Calculating OK
generating timestep
Calculating OK
generating timestep
Calculating OK
generating timestep
Calculating OK
generating timestep
Calculating OK
generating timestep
Calculating OK
generating timestep
Calculating OK
generating timestep
Calculating OK
generating timestep
Calculating OK
generating timestep


In [26]:
time_k

[([20.347001552488095,
   20.34694054948418,
   20.346879414532804,
   20.351601747158302,
   20.3515408613529,
   20.356141068220666,
   20.3606803867291,
   20.36528050066708,
   20.365219702683095,
   20.369880509381453,
   20.36981982878853,
   20.37454084376353,
   20.374480412776258,
   20.374419849639537,
   20.38049124590505,
   20.38043397324038,
   20.38037656834697,
   20.380319031226747,
   20.380261361881633,
   20.38020356031355,
   20.380145626524442,
   20.380087560516245,
   20.380029362290898,
   20.379200705433906,
   20.379140524187456,
   20.379080210755543,
   20.385202017837194,
   20.385145128104938,
   20.385088106106217,
   20.385030951842936,
   20.384568956442973,
   20.384510611885247,
   20.38445213508228,
   20.384393526036035,
   20.384334784748468,
   20.38427591122155,
   20.384216905457244,
   20.383979560065722,
   20.38391989314419,
   20.383860093997182,
   20.383800162626702,
   20.38374009903475,
   20.38979853383294,
   20.389741762634994,
   20

In [15]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Set the default renderer to 'browser'
pio.renderers.default = 'browser'

# Create a subplot with mapbox
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scattermapbox'}]])

# Add the initial densitymapbox trace
initial_density_lat, initial_density_lon, initial_density_z = time_k[0]
fig.add_trace(go.Densitymapbox(lat=initial_density_lat, lon=initial_density_lon,
                               z=initial_density_z, colorscale='Jet', showscale=True,
                               zmin=np.min(z_masked), zmax=np.max(z_masked),
                               customdata=initial_density_z,
                               hovertemplate='%{customdata:.2f} NPM/100ml',
                               name='Coliformes totales', radius=15), row=1, col=1)

# Add the scatter plot data as a scattermapbox trace
fig.add_trace(go.Scattermapbox(lat=lats, lon=lons, mode='markers',
                               marker=dict(color='violet', size=15),
                               text=values, hovertemplate='%{text:.2f} NPM/100ml',
                               name='Data Points'), row=1, col=1)

# Set the mapbox configuration
fig.update_layout(mapbox=dict(style='open-street-map', zoom=10,
                              center=dict(lat=np.mean([min(lats), max(lats)]),
                                          lon=np.mean([min(lons), max(lons)]))),
                  title='OK L2 - Linear',
                  margin=dict(l=20, r=20, t=80, b=20),
                  # Add animation settings
                  updatemenus=[dict(type='buttons', showactive=False,
                                    buttons=[dict(label='Play',
                                                  method='animate',
                                                  args=[None, {'frame': {'duration': 300, 'redraw': True},
                                                               'fromcurrent': True}])])])

# Initialize the frames list
frames = []

# Create frames for each time step
for density_lat, density_lon, density_z in time_k:
    frame = go.Frame(data=[go.Densitymapbox(lat=density_lat, lon=density_lon,
                                            z=density_z, colorscale='Jet', showscale=True,
                                            zmin=np.min(z_masked), zmax=np.max(z_masked),
                                            customdata=density_z,
                                            hovertemplate='%{customdata:.2f} NPM/100ml',
                                            name='Coliformes totales', radius=15)])
    frames.append(frame)

# Add frames to the figure
fig.frames = frames

# Show the figure
fig.show()


(exo-open:4796): Gtk-WARNING **: 04:56:35.620: Theme parsing error: gtk-contained-dark.css:2871:228: Missing closing bracket for :not()

(xfce4-mime-helper:4801): Gtk-WARNING **: 04:56:35.804: Theme parsing error: gtk-contained-dark.css:2871:228: Missing closing bracket for :not()

(firefox:4805): Gtk-WARNING **: 04:56:35.981: Theme parsing error: gtk-contained-dark.css:2871:228: Missing closing bracket for :not()
